In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json

In [2]:
dirs4 = [(0, 1), (0, -1), (1, 0), (-1, 0)]
dirs8 = dirs4 + [(1, 1), (1, -1), (-1, 1), (-1, -1)]

In [3]:
boss = {}
shop = {}

with open("./data/day21.txt") as f:
    while line := f.readline():
        line = line.rstrip().split(': ')
        boss[line[0]] = int(line[1])

with open("./data/day21_ex.txt") as f:
    blocks = f.read().rstrip().split('\n\n')

    shop['Weapons'] = {}
    for item in blocks[0].split('\n')[1:]:
        item = item.split()
        shop['Weapons'][item[0]] = {'Cost':int(item[1]), 'Damage':int(item[2]), 'Armor':int(item[3])}
    
    shop['Armor'] = {}
    for item in blocks[1].split('\n')[1:]:
        item = item.split()
        shop['Armor'][item[0]] = {'Cost':int(item[1]), 'Damage':int(item[2]), 'Armor':int(item[3])}

    shop['Rings'] = {}
    for item in blocks[2].split('\n')[1:]:
        item = item.split()
        shop['Rings'][item[0]+' '+item[1]] = {'Cost':int(item[2]), 'Damage':int(item[3]), 'Armor':int(item[4])}

shop

{'Weapons': {'Dagger': {'Cost': 8, 'Damage': 4, 'Armor': 0},
  'Shortsword': {'Cost': 10, 'Damage': 5, 'Armor': 0},
  'Warhammer': {'Cost': 25, 'Damage': 6, 'Armor': 0},
  'Longsword': {'Cost': 40, 'Damage': 7, 'Armor': 0},
  'Greataxe': {'Cost': 74, 'Damage': 8, 'Armor': 0}},
 'Armor': {'Leather': {'Cost': 13, 'Damage': 0, 'Armor': 1},
  'Chainmail': {'Cost': 31, 'Damage': 0, 'Armor': 2},
  'Splintmail': {'Cost': 53, 'Damage': 0, 'Armor': 3},
  'Bandedmail': {'Cost': 75, 'Damage': 0, 'Armor': 4},
  'Platemail': {'Cost': 102, 'Damage': 0, 'Armor': 5}},
 'Rings': {'Damage +1': {'Cost': 25, 'Damage': 1, 'Armor': 0},
  'Damage +2': {'Cost': 50, 'Damage': 2, 'Armor': 0},
  'Damage +3': {'Cost': 100, 'Damage': 3, 'Armor': 0},
  'Defense +1': {'Cost': 20, 'Damage': 0, 'Armor': 1},
  'Defense +2': {'Cost': 40, 'Damage': 0, 'Armor': 2},
  'Defense +3': {'Cost': 80, 'Damage': 0, 'Armor': 3}}}

In [4]:
def add_dicts(ls):
    res = ls[0].copy()
    for d in ls[1:]:
        for k, v in d.items():
            res[k] += v
    return res

In [5]:
def play(stats):
    player_hp = 100
    boss_hp = boss['Hit Points']
    player_dmg = max(0, stats['Damage']-boss['Armor'])
    boss_dmg = max(0, boss['Damage']-stats['Armor'])
    if player_dmg == 0:
        return False
    if boss_dmg == 0:
        return True
    while player_hp > 0 and boss_hp > 0:
        boss_hp -= player_dmg
        if boss_hp <= 0:
            return True
        player_hp -= boss_dmg
        if player_hp <= 0:
            return False
    raise Exception("How did we end up here??", stats)

In [6]:
play(add_dicts([
    shop['Weapons']['Dagger'],
    shop['Armor']['Platemail'],
    shop['Rings']['Damage +3']
]))

True

In [7]:
play(add_dicts([
    shop['Weapons']['Dagger'],
    shop['Armor']['Platemail'],
    shop['Rings']['Defense +1']
]))

False

In [8]:
res = []
for wep in shop['Weapons'].values():
    for arm in [{}] + list(shop['Armor'].values()):
        d = add_dicts([wep, arm])
        if play(d):
            res.append(d['Cost'])
        rings = list(shop['Rings'].values())
        for i, r1 in enumerate(rings):
            d = add_dicts([wep, arm, r1])
            if play(d):
                res.append(d['Cost'])
            for r2 in rings[i+1:]:
                d = add_dicts([wep, arm, r1, r2])
                if play(d):
                    res.append(d['Cost'])
min(res)

121

In [9]:
res = []
for wep in shop['Weapons'].values():
    for arm in [{}] + list(shop['Armor'].values()):
        d = add_dicts([wep, arm])
        if not play(d):
            res.append(d['Cost'])
        rings = list(shop['Rings'].values())
        for i, r1 in enumerate(rings):
            d = add_dicts([wep, arm, r1])
            if not play(d):
                res.append(d['Cost'])
            for r2 in rings[i+1:]:
                d = add_dicts([wep, arm, r1, r2])
                if not play(d):
                    res.append(d['Cost'])
max(res)

201